In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import optuna


In [2]:

# Load features and labels
features = pd.read_csv('nn_features.csv', index_col=0).values
labels = pd.read_csv('nn_labels.csv', index_col=0).values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:

# Customizable model class
'''class PitchVelocityModel(nn.Module):
    def __init__(self, num_features, hidden_units):
        super(PitchVelocityModel, self).__init__()
        layers = []
        for i in range(len(hidden_units)):
            if i == 0:
                layers.append(nn.Linear(num_features, hidden_units[i]))
            else:
                layers.append(nn.Linear(hidden_units[i-1], hidden_units[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_units[-1], 1))  # Output layer
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x)'''

class PitchVelocityModel(nn.Module):
    def __init__(self, num_features):
        super(PitchVelocityModel, self).__init__()
        # Fully connected layers
        self.fc1 = torch.nn.Linear(num_features, 1024)
        self.fc2 = torch.nn.Linear(1024, 512)
        self.fc3 = torch.nn.Linear(512, 256)
        self.fc4 = torch.nn.Linear(256, 128)
        self.fc5 = torch.nn.Linear(128, 64)
        self.fc6 = torch.nn.Linear(64, 32)
        self.fc7 = torch.nn.Linear(32, 1)
        
        # Relu activation function
        self.relu = torch.nn.ReLU()  
                
    def forward(self, x):
        # Forward pass
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.relu(self.fc6(x))
        x = self.fc7(x)
        
        return x

In [4]:

# Objective function for Optuna
def objective(trial):
    # Hyperparameters to be tuned
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    #num_layers = trial.suggest_int('num_layers', 1, 4)
    #hidden_units = [trial.suggest_int(f'n_units_l{i}', 16, 128) for i in range(num_layers)]

    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Model setup
    model = PitchVelocityModel(X_train.shape[1])
    model.to(device)

    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    epochs=100

    # Training loop
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)  # Both y_pred and y_batch are now [batch_size, 1]
            loss.backward()
            optimizer.step()


    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            val_loss += criterion(y_pred, y_batch).item()  # Both y_pred and y_batch are now [batch_size, 1]

    return val_loss / len(val_loader)


In [5]:

# Creating the Optuna study object
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Best hyperparameters
best_params = study.best_params
print('Best trial:', best_params)


[I 2024-02-13 14:46:36,539] A new study created in memory with name: no-name-015aa066-25a8-4596-9f58-75b2001cf940


[I 2024-02-13 14:49:33,930] Trial 0 finished with value: 93.50797895284799 and parameters: {'batch_size': 32, 'lr': 0.04334623097187218}. Best is trial 0 with value: 93.50797895284799.
[I 2024-02-13 14:51:26,625] Trial 1 finished with value: 22.198329210281372 and parameters: {'batch_size': 64, 'lr': 0.0029369756151750477}. Best is trial 1 with value: 22.198329210281372.
[I 2024-02-13 14:52:53,146] Trial 2 finished with value: 23.058248043060303 and parameters: {'batch_size': 64, 'lr': 0.003551628179648043}. Best is trial 1 with value: 22.198329210281372.
[I 2024-02-13 14:53:46,504] Trial 3 finished with value: 4637.768047626202 and parameters: {'batch_size': 128, 'lr': 0.04129969560895682}. Best is trial 1 with value: 22.198329210281372.
[I 2024-02-13 14:54:37,015] Trial 4 finished with value: 36.12224116692176 and parameters: {'batch_size': 128, 'lr': 0.00034544462444132567}. Best is trial 1 with value: 22.198329210281372.
[I 2024-02-13 14:56:09,992] Trial 5 finished with value: 23.1

Best trial: {'batch_size': 32, 'lr': 0.005221631343320849}


: 